In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

iris = load_iris()
X,y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
joblib.dump(clf, 'output/model.pkl', compress=9)

['output/model.pkl']

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import joblib

"""FloatTensorType with a shape of [None, 4]. 
The None in the shape indicates that the batch size is variable,
and 4 represents the number of features in the input data."""

cls = joblib.load('output/model.pkl')
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(cls, initial_types=initial_type)

with open('output/model.onnx', 'wb') as f:
    f.write(onx.SerializeToString())

In [ ]:
import onnxruntime as rt
import numpy as np

data = np.array([[4.5,4.9,5.1,5.4],[1.5,2.9,3.1,1.4],[7.5,6.9,8.1,6.4]])

sess = rt.InferenceSession("./output/model.onnx")
input_name = sess.get_inputs()[0].name # Return the inputs metadata as a list 
label_name = sess.get_outputs()[0].name # Return the outputs metadata as a list

pred_onnx = sess.run([label_name], {input_name: data.astype(np.float32)})[0]
print(pred_onnx)

[2 1 2]